# Long Short Term memory and other Neural Networks

In [ ]:
import duckdb
import pandas as pd
import pandas_ta as ta
import os
import csv
import itertools


def trail_stoploss(row, tradingdata, whole_percentage):
    idx = row.name
    td = tradingdata
    pct = whole_percentage/100
    max_holdtime = 30 # Only hold the trade for twenty minutes

    after_td = td.loc[idx:idx+max_holdtime+1] 
    
    basis = td['close'].loc[idx]
    max_price = basis   # Initially the max price is the basis - No Shorting to Start
    min_price = basis - (basis*pct)
    time_stop = idx + max_holdtime 
    time_counter = idx
    
    #print(f"Basis: {basis}")

    for i, row in after_td.iterrows():
        close = row['close']
        #print(f"Close: {close}, Max: {max_price}, Min: {min_price}, Counter: {time_counter-idx}")
        if close > max_price:
            max_price = close
            min_price = max_price - (max_price * pct)
        elif close < min_price:
            profit = (close - basis) / basis
            return [i, profit]
        
        time_counter += 1
        if time_counter > time_stop:
            break

    # If the loop ends without triggering stop loss, calculate the profit based on the last close
    profit = (close - basis) / basis
    return profit


con = duckdb.connect(r"C:\Users\rybot\OneDrive\Databases\datadump.duckdb")

# Filter data for January to March 2022
start_date = '2022-01-01'
end_date = '2022-03-31'

qry_month = f'''
    SELECT * FROM Stocks
    WHERE Datetime >= '{start_date}' 
    AND Datetime < '{end_date}' 
    AND Stock = 'AAPL' 
    AND Interval = 1
'''
df = con.execute(qry_month).fetchdf()
# Sort the Data from oldest to newest
df = df.sort_values(by='Datetime').reset_index(drop=True)

# List of Testing Variables
Data_order = ["Random","Sorted"] # *combination[3]
#Underlying_data = ["Percent Change","Raw Data"]
Field_size = [10,20,50,100,200,300] # *combination[2]
Derivatives = [["dydx"],["dydx","dydx2"]] # *combination[0]
TAs = [[],["EMA"],["EMA","SMA"]] # *combination[1]

all_combinations = itertools.product(Derivatives, TAs, Field_size, Data_order)

# Getting all of the trading days because trades can only occur during trading day
df['Time'] = df['Datetime'].dt.time
start_time = pd.to_datetime('09:00:00').time()
end_time = pd.to_datetime('16:00:00').time()
TradingDay_df = df[(df['Time'] > start_time) & (df['Time'] < end_time) & (df.index >= 201)]

df['Profit'] = TradingDay_df.apply(trail_stoploss, axis=1, tradingdata=df,whole_percentage=5)

#applying EMAs SMAs
EMAs = [8,10,20,50,75,100]
SMAs = [8,10,20,50,75,100]


for EMA in EMAs:
    df[f'EMA{EMA}'] = ta.ema(df['close'], length=EMA)

for SMA in SMAs:
    df[f'SMA{SMA}'] = ta.sma(df['close'], length=SMA)

# 1st
df['Time_Diff'] = df['Datetime'].diff().dt.total_seconds()
df['dydx'] = df['close'].diff() / df['Time_Diff']

# 2nd
df['dydx2'] = df['dydx'].diff() / df['Time_Diff']

print(df.describe())


In [20]:
qry = '''
    SELECT DISTINCT(STOCK)
    FROM STOCKS
'''

#df3w = con.execute(qry).fetchdf()
#print(df3w)

ddf = pd.read_excel(r"C:\Design Folder\RBGithub\Financial-Forecasting\Characteristics of Test Data Database.xlsx")
print(ddf.info())

lists = list(ddf['year'])
ticklist = list(ddf['Ticker'])

for Yr in lists:
    for ticker in ticklist:
        qry = f''' 
            SELECT FIRST(Ticker) AS Tick, MAX(Max_Price) AS Max, MIN(Min_Price) AS Min, MIN(Average_Price) AS Avg
            FROM ddf
            WHERE year = {Yr} and Ticker = {ticker}
        '''
        dfg = duckdb.query(qry).df()
        print(f"Year = {Yr}, \n {dfg}")



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Ticker         240 non-null    object 
 1   year           240 non-null    int64  
 2   No_Records     240 non-null    int64  
 3   Min_Price      240 non-null    float64
 4   Max_Price      240 non-null    float64
 5   Average_Price  240 non-null    float64
dtypes: float64(3), int64(2), object(1)
memory usage: 11.4+ KB
None


BinderException: Binder Error: Referenced column "AAPL" not found in FROM clause!
Candidate bindings: "ddf.year"

In [4]:
import pandas as pd
import duckdb
con = duckdb.connect(r"C:\Users\rybot\OneDrive\Databases\datadump.duckdb")

# Filter data for January to March 2022
start_date = '2022-01-01'
end_date = '2022-03-31'

qry_month = f'''
    SELECT * FROM Stocks
    WHERE Datetime >= '{start_date}' 
    AND Datetime < '{end_date}' 
    AND Stock = 'AAPL' 
    AND Interval = 1
'''
df = con.execute(qry_month).fetchdf()
# Sort the Data from oldest to newest
df = df.sort_values(by='Datetime').reset_index(drop=True)

with pd.ExcelWriter(r"C:\Users\rybot\OneDrive\Databases\2022_m1to3_appl.xlsx", engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='Sheet1', index=False)